# Sequence example

>https://github.com/dennybritz/tf-rnn/blob/master/sequence_example.ipynb

In [11]:
import tensorflow as tf
import numpy as np

tfrecords_filename= 'sequence_examples.tfrecords'

In [12]:
sequences= [[1,2,3], [4,5,1], [1,2]]
label_sequences= [[0,1,0], [1,0,0], [1,1]]

def make_example(sequences, labels):
    # The object we return
    ex= tf.train.SequenceExample() 
    # A non-sequential feature of our example
    sequence_length=len(sequences)
    ex.context.feature['length'].int64_list.value.append(sequence_length)
    #Feature lists for the two sequential feature of our example
    f1_tokens= ex.feature_lists.feature_list['tokens']
    f1_labels= ex.feature_lists.feature_list['labels']
    for token, label in zip(sequence, labels):
        f1_tokens.feature.add().int64_list.value.append(token)
        f1_labels.feature.add().int64_list.value.append(label)
    return ex

#Write all examples into a TFRecords file
with open(tfrecords_filename) as f:
    writer= tf.python_io.TFRecordWriter(f.name)
    for sequence, label_sequence in zip(sequences, label_sequences):
        ex= make_example(sequence, label_sequence)
        print(ex.SerializeToString())
        writer.write(ex.SerializeToString())        
    writer.close()
    print('Wrote to {}'.format(f.name))


b'\n\x11\n\x0f\n\x06length\x12\x05\x1a\x03\n\x01\x03\x12B\n\x1f\n\x06tokens\x12\x15\n\x05\x1a\x03\n\x01\x01\n\x05\x1a\x03\n\x01\x02\n\x05\x1a\x03\n\x01\x03\n\x1f\n\x06labels\x12\x15\n\x05\x1a\x03\n\x01\x00\n\x05\x1a\x03\n\x01\x01\n\x05\x1a\x03\n\x01\x00'
b'\n\x11\n\x0f\n\x06length\x12\x05\x1a\x03\n\x01\x03\x12B\n\x1f\n\x06tokens\x12\x15\n\x05\x1a\x03\n\x01\x04\n\x05\x1a\x03\n\x01\x05\n\x05\x1a\x03\n\x01\x01\n\x1f\n\x06labels\x12\x15\n\x05\x1a\x03\n\x01\x01\n\x05\x1a\x03\n\x01\x00\n\x05\x1a\x03\n\x01\x00'
b'\n\x11\n\x0f\n\x06length\x12\x05\x1a\x03\n\x01\x02\x124\n\x18\n\x06tokens\x12\x0e\n\x05\x1a\x03\n\x01\x01\n\x05\x1a\x03\n\x01\x02\n\x18\n\x06labels\x12\x0e\n\x05\x1a\x03\n\x01\x01\n\x05\x1a\x03\n\x01\x01'
Wrote to sequence_examples.tfrecords


In [34]:
#reading back
record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

for string_record in record_iterator:
    ex = tf.train.SequenceExample()
    ex.ParseFromString(string_record)
#     print(ex)
    #Length of the sequence
    print(ex.context.feature['length'].int64_list.value[0])
    #First element of every label
    print(ex.feature_lists.feature_list['labels'].feature[0].int64_list.value[0])
    #Last element of every token
    print(ex.feature_lists.feature_list['tokens'].feature[0].int64_list.value[-1])
    

3
0
1
3
1
4
2
1
1


In [6]:
tf.reset_default_graph()

# A single serialized example
#(You can read this from a file using TFRecordReader)
ex= make_example([1, 2, 3], [0, 1, 0]).SerializeToString() 

#Define how to parse the example
context_features= {
    'length': tf.FixedLenFeature([], dtype=tf.int64)
}
sequence_features= {
    'tokens': tf.FixedLenSequenceFeature([], dtype=tf.int64),
    'labels': tf.FixedLenSequenceFeature([], dtype=tf.int64)
}

#Parse the example (returns a dictionary of tensors)
context_parsed, sequence_parsed= tf.parse_single_sequence_example(
    serialized=ex,
    context_features=context_features,
    sequence_features=sequence_features
)
context= tf.contrib.learn.run_n(context_parsed, n=1, )

Instructions for updating:
graph_actions.py will be deleted. Use tf.train.* utilities instead. You can use learn/estimators/estimator.py as an example.
Instructions for updating:
graph_actions.py will be deleted. Use tf.train.* utilities instead. You can use learn/estimators/estimator.py as an example.
Instructions for updating:
graph_actions.py will be deleted. Use tf.train.* utilities instead. You can use learn/estimators/estimator.py as an example.


In [46]:
#reading back
record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

#Define how to parse the example
context_features= {
    'length': tf.FixedLenFeature([], dtype=tf.int64)
}
sequence_features= {
    'tokens': tf.FixedLenSequenceFeature([], dtype=tf.int64),
    'labels': tf.FixedLenSequenceFeature([], dtype=tf.int64)
}

for string_record in record_iterator:
#     ex= tf.train.SequenceExample()
#     ex.ParseFromString(string_record)
    
    #Parse the example (returns a dictionary of tensors)
    context_parsed, sequence_parsed= tf.parse_single_sequence_example(
        serialized=string_record,
        context_features=context_features,
        sequence_features=sequence_features
    )
#     import code; code.interact(local=dict(globals(), **locals()))
    print(context_parsed)
    print(sequence_parsed)

{'length': <tf.Tensor 'ParseSingleSequenceExample_8/ParseSingleSequenceExample:0' shape=() dtype=int64>}
{'labels': <tf.Tensor 'ParseSingleSequenceExample_8/ParseSingleSequenceExample:1' shape=(?,) dtype=int64>, 'tokens': <tf.Tensor 'ParseSingleSequenceExample_8/ParseSingleSequenceExample:2' shape=(?,) dtype=int64>}
{'length': <tf.Tensor 'ParseSingleSequenceExample_9/ParseSingleSequenceExample:0' shape=() dtype=int64>}
{'labels': <tf.Tensor 'ParseSingleSequenceExample_9/ParseSingleSequenceExample:1' shape=(?,) dtype=int64>, 'tokens': <tf.Tensor 'ParseSingleSequenceExample_9/ParseSingleSequenceExample:2' shape=(?,) dtype=int64>}
{'length': <tf.Tensor 'ParseSingleSequenceExample_10/ParseSingleSequenceExample:0' shape=() dtype=int64>}
{'labels': <tf.Tensor 'ParseSingleSequenceExample_10/ParseSingleSequenceExample:1' shape=(?,) dtype=int64>, 'tokens': <tf.Tensor 'ParseSingleSequenceExample_10/ParseSingleSequenceExample:2' shape=(?,) dtype=int64>}
